解压原始数据。

In [1]:
import os
import tarfile
import gzip
import shutil

def gunzip(file_path):
    file = file_path.replace('.gz', '')
    g_file = gzip.GzipFile(file_path)
    with open(file, 'wb+') as f:
        f.write(g_file.read())
    g_file.close()


def untar(file_path, out_root):
    file_name = os.path.split(file_path)[-1]
    file = file_name.replace('.tar', '')
    out_path = os.path.join(out_root, file)
    if not os.path.exists(out_path):
        os.makedirs(out_path)
    tar = tarfile.open(file_path)
    for sub in tar.getnames():
        print(sub)
        tar.extract(sub, out_path)
    tar.close()

    # move file into out_path, and remove other dir in out_path
    for a, b, c in os.walk(out_path):
        if a == out_path:
            continue
        for i in c:
            tmp_path = os.path.join(a, i)
            shutil.move(tmp_path, out_path)
        os.removedirs(a)

gunzip(r'Test\20190221_T01_ZHANGZJ_LX_LIUZHONGXUN.tar.gz')
untar(r'Test\20190221_T01_ZHANGZJ_LX_LIUZHONGXUN.tar', r'Test')

ZHANGZJ_LX_LIUZHONGXUN.MR.BNU_CST_2011_ZHANG_ZHANJUN.0002.0088.2019.02.21.10.03.25.687500.107967098.IMA
20190221_T01_ZHANGZJ_LX_LIUZHONGXUN/20190221_T01_ZHANGZJ_LX_LIUZHONGXUN.MR.BNU_CST_2011_ZHANG_ZHANJUN.0003.0126.2019.02.21.10.03.25.687500.107972888.IMA
20190221_T01_ZHANGZJ_LX_LIUZHONGXUN/20190221_T01_ZHANGZJ_LX_LIUZHONGXUN.MR.BNU_CST_2011_ZHANG_ZHANJUN.0001.0002.2019.02.21.10.03.25.687500.107962460.IMA
20190221_T01_ZHANGZJ_LX_LIUZHONGXUN/20190221_T01_ZHANGZJ_LX_LIUZHONGXUN.MR.BNU_CST_2011_ZHANG_ZHANJUN.0002.0207.2019.02.21.10.03.25.687500.107969258.IMA
20190221_T01_ZHANGZJ_LX_LIUZHONGXUN/20190221_T01_ZHANGZJ_LX_LIUZHONGXUN.MR.BNU_CST_2011_ZHANG_ZHANJUN.0006.0004.2019.02.21.10.03.25.687500.107976842.IMA
20190221_T01_ZHANGZJ_LX_LIUZHONGXUN/20190221_T01_ZHANGZJ_LX_LIUZHONGXUN.MR.BNU_CST_2011_ZHANG_ZHANJUN.0006.0023.2019.02.21.10.03.25.687500.107977191.IMA
20190221_T01_ZHANGZJ_LX_LIUZHONGXUN/20190221_T01_ZHANGZJ_LX_LIUZHONGXUN.MR.BNU_CST_2011_ZHANG_ZHANJUN.0003.0140.2019.02.21.10.03.25


分解模态。

In [2]:
import os
import shutil
import nibabel as nib
import pydicom

def sorted(file_path):
    file_name = os.path.split(file_path)[-1]
    for i in os.listdir(file_path):
        dcm_file = os.path.join(file_path, i)
        ds = pydicom.dcmread(dcm_file)
        series_name = ds[0x0018, 0x1030].value
        series_name = series_name.replace('.', '_')
        series_id = ds[0x0020, 0x0011].value
        out_path = os.path.join(file_path, f'{int(series_id):04d}_{series_name}')
        if not os.path.exists(out_path):
            os.makedirs(out_path)
        shutil.move(os.path.join(file_path, i), out_path)

sorted(r'Test\20190221_T01_ZHANGZJ_LX_LIUZHONGXUN')

将不同模态数据复制到相应的文件夹。

In [1]:
import os
import shutil
import tarfile

def tar(file_path, out_file):
    tar = tarfile.open(out_file, 'w:gz')
    for a, b, c in os.walk(file_path):
        for i in c:
            tar.add(os.path.join(a, i), arcname=i)
    tar.close()

def mv(file_path, out_root, machine='OLD'):
    file_name = os.path.split(file_path)[-1]
    
    squences = {
            'ge_func_31x31x35_240_RS': "REST",
            'ge_func_3x3x3p5_303': "EM",
            'ge_func_3x3x3p5_235': "NBACK",
            't1_mprage_sag_tr1900': "T1",
            't2_tirm_tra_dark-fluid-p2_new': "FLAIR",
            'DTI_30_2x2x2_128x128_b1000_p2': "DTI",
            't2_tse_tra_320_p2': "T2",
            'ge_func_3x3x3p5_360': "LANT"
            }
    if machine == 'OLD':
        pass
    else:
        squences = {
            't1_mprage_sag_1x1x1_p2_20ch': "T1",
            'bold_64x64_s2_3x3x3p5_tr1000_fa70': "REST",
            'bold_80x80_s2_2.5iso_tr2000': "REST",
            'DTI_BP_64_2x2x2_s2_p2_6-8': "DTI",
            'field_mapping_2x2x2_for_DTI': "DTI",
            'AAHead_Scout': "AAHead_Scout",
            't2_blade_tra_p2': "T2WI",
            't1_tirm_tra_dark-fluid_p2': "FLAIR",
            't2_tirm_tra_dark-fluid': "FLAIR",
            't2_blade_sag': "T2WI",
            'HighResHippocampus': "HIPPO",
            'TOF_3D_multi-slab': "TOF",
            'Sagittal 3D FLAIR': "3D-FLAIR",
            'ASL_3D_tra_iso': "ASL"
        }

    for i in os.listdir(file_path):
        series_name = i[5:]
        if series_name in squences.keys():
            tmp_src_path = os.path.join(file_path, i)
            tmp_dst_path = os.path.join(out_root, squences[series_name], file_name)
            if not os.path.exists(tmp_dst_path):
                os.makedirs(tmp_dst_path)
            for j in os.listdir(tmp_src_path): 
                shutil.copy(os.path.join(tmp_src_path, j), os.path.join(tmp_dst_path, j))
            tar(tmp_dst_path, f'{tmp_dst_path}.tar.gz')
            shutil.rmtree(tmp_dst_path)
    shutil.rmtree(file_path)


mv(r'Test\20190221_T01_ZHANGZJ_LX_LIUZHONGXUN', 'Test', 'OLD')

In [1]:
import pandas as pd

oldDf = pd.read_excel(r'0.List\师大核磁人员情况新_20200115.xls', sheet_name='Sheet1', header=0)
oldDf['CheckDateName'] = ['no']*len(oldDf)
oldDf

,序号,姓名,性别,年龄,教育,社区,时间,身份证号,备注,结构,...,情景记忆,工作记忆,DTI,flair,临床,失独老人任务,CheckDateName,CheckDateName_A,CheckSeries,CheckSeries_A
0,2514,褚运,男,67,11.0,王建平,20200104,NaN,NaN,√,...,NaN,NaN,√,√,NaN,√,no,NaN,NaN,NaN
1,2515,丛敏,女,63,13.0,王建平,20200104,1.101062e+17,NaN,√,...,NaN,NaN,√,√,NaN,√,no,NaN,NaN,NaN
2,2516,张淑琴,女,59,12.0,王建平,20200104,1.101012e+17,NaN,√,...,NaN,NaN,√,√,NaN,√,no,NaN,NaN,NaN
3,2517,陶晓坤,女,62,12.0,王建平,20200104,2.327002e+17,NaN,√,...,NaN,NaN,√,√,NaN,√,no,NaN,NaN,NaN
4,2518,张美伶,女,55,8.0,王建平,20200104,1.102282e+17,NaN,√,...,NaN,NaN,√,√,NaN,√,no,NaN,NaN,NaN
5,2519,王金凤,女,62,12.0,王建平,20200104,1.101082e+17,NaN,√,...,NaN,NaN,√,√,NaN,√,no,NaN,NaN,NaN
6,2520,何婷,女,62,15.5,安德里,20200107,1.101012e+17,25182-2,√,...,NaN,NaN,√,√,NaN,√,no,NaN,NaN,NaN
7,2521,窦建国,男,63,14.5,安德里,20200107,1.101012e+17,25561-1,√,...,√,√,√,√,NaN,NaN,no,NaN,NaN,NaN
8,2522,徐宏,女,67,13.0,安德里,20200107,1.101052e+17,25585-1 2颗纯钛种植牙，下左5 右5,√,...,√,√,√,√,NaN,NaN,no,NaN,NaN,NaN
9,2523,张燕,女,63,9.0,王家园,20200107,1.101012e+17,NaN,√,...,√,√,√,√,NaN,NaN,no,NaN,NaN,NaN


In [12]:
a = oldDf.loc[1, '结构']
a

&#39;√&#39;

In [13]:
import numpy as np

if a != '√':
    print('ss')